In [1]:
import pandas as pd
import numpy as np
import simplejson

from math import sqrt

from Arquivos.Base_de_Dados import avaliacoes as avaliacoesUsuario

In [2]:
avaliacoesUsuario

{'Ana': {'Freddy x Jason': 2.5,
  'O Ultimato Bourne': 3.5,
  'Star Trek': 3.0,
  'Exterminador do Futuro': 3.5,
  'Norbit': 2.5,
  'Star Wars': 3.0},
 'Marcos': {'Freddy x Jason': 3.0,
  'O Ultimato Bourne': 3.5,
  'Star Trek': 1.5,
  'Exterminador do Futuro': 5.0,
  'Star Wars': 3.0,
  'Norbit': 3.5},
 'Pedro': {'Freddy x Jason': 2.5,
  'O Ultimato Bourne': 3.0,
  'Exterminador do Futuro': 3.5,
  'Star Wars': 4.0},
 'Claudia': {'O Ultimato Bourne': 3.5,
  'Star Trek': 3.0,
  'Star Wars': 4.5,
  'Exterminador do Futuro': 4.0,
  'Norbit': 2.5},
 'Adriano': {'Freddy x Jason': 3.0,
  'O Ultimato Bourne': 4.0,
  'Star Trek': 2.0,
  'Exterminador do Futuro': 3.0,
  'Star Wars': 3.0,
  'Norbit': 2.0},
 'Janaina': {'Freddy x Jason': 3.0,
  'O Ultimato Bourne': 4.0,
  'Star Wars': 3.0,
  'Exterminador do Futuro': 5.0,
  'Norbit': 3.5},
 'Leonardo': {'O Ultimato Bourne': 4.5,
  'Norbit': 1.0,
  'Exterminador do Futuro': 4.0}}

In [3]:
df_evaluation = pd.DataFrame(avaliacoesUsuario).T
df_evaluation.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7 entries, Ana to Leonardo
Data columns (total 6 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Freddy x Jason          5 non-null      float64
 1   O Ultimato Bourne       7 non-null      float64
 2   Star Trek               4 non-null      float64
 3   Exterminador do Futuro  7 non-null      float64
 4   Norbit                  6 non-null      float64
 5   Star Wars               6 non-null      float64
dtypes: float64(6)
memory usage: 392.0+ bytes


In [4]:
df_evaluation

,Freddy x Jason,O Ultimato Bourne,Star Trek,Exterminador do Futuro,Norbit,Star Wars
Ana,2.5,3.5,3.0,3.5,2.5,3.0
Marcos,3.0,3.5,1.5,5.0,3.5,3.0
Pedro,2.5,3.0,NaN,3.5,NaN,4.0
Claudia,NaN,3.5,3.0,4.0,2.5,4.5
Adriano,3.0,4.0,2.0,3.0,2.0,3.0
Janaina,3.0,4.0,NaN,5.0,3.5,3.0
Leonardo,NaN,4.5,NaN,4.0,1.0,NaN


---

In [5]:
def euclidiana(base, usuario1, usuario2):
    si = {}
    for item in base[usuario1]:
        if item in base[usuario2]: si[item] = 1
    
    if len(si) == 0: return 0
    
    soma = sum([pow(base[usuario1][item] - base[usuario2][item], 2)
               for item in base[usuario1] if item in base[usuario2]])
    return 1/(1 + sqrt(soma))

In [6]:
euclidiana(avaliacoesUsuario, 'Leonardo', 'Ana')

0.3483314773547883

---

In [7]:
def getSimilares(base, usuario):
    similaridade = [(euclidiana(base, usuario, outro), outro)
                   for outro in base if outro != usuario]
    
    similaridade.sort(reverse=True)
    return similaridade[0:30]

In [8]:
getSimilares(avaliacoesUsuario, 'Ana')

[(0.4721359549995794, 'Pedro'),
 (0.4142135623730951, 'Adriano'),
 (0.38742588672279304, 'Claudia'),
 (0.3483314773547883, 'Leonardo'),
 (0.3405424265831667, 'Janaina'),
 (0.29429805508554946, 'Marcos')]

---

In [9]:
def getRecomendacoesUsuario(base, usuario):
    totais = {}
    somaSimilaridade = {}
    
    for outro in base:
        if outro == usuario: continue
        similaridade = euclidiana(base, usuario, outro)
        
        if similaridade <= 0: continue
        
        for item in base[outro]:
            if item not in base[usuario]:
                totais.setdefault(item, 0)
                totais[item] += base[outro][item] * similaridade
                somaSimilaridade.setdefault(item, 0)
                somaSimilaridade[item] += similaridade
    
    rankings = [(total / somaSimilaridade[item], item) for item, total in totais.items()]
    rankings.sort(reverse=True)
    return rankings[0:30]

In [10]:
getRecomendacoesUsuario(avaliacoesUsuario, 'Leonardo')

[(3.457128694491423, 'Star Wars'),
 (2.778584003814924, 'Freddy x Jason'),
 (2.422482042361917, 'Star Trek')]

In [11]:
getRecomendacoesUsuario(avaliacoesUsuario, 'Pedro')

[(2.4825817086405517, 'Star Trek'), (2.453379230569188, 'Norbit')]

In [12]:
getRecomendacoesUsuario(avaliacoesUsuario, 'Claudia')

[(2.74908990418004, 'Freddy x Jason')]

In [13]:
getRecomendacoesUsuario(avaliacoesUsuario, 'Janaina')

[(2.203053498033553, 'Star Trek')]

---

In [14]:
# Colocar os filme nos lugares dos nomes (json)
avaliacoesFilme = df_evaluation.to_dict()
avaliacoesFilme = simplejson.loads(simplejson.dumps(avaliacoesFilme, ignore_nan=True))

In [15]:
# Remover valores vazios
venone = []
movies = list(avaliacoesFilme.keys())

for i in range(0, len(movies)):
    for k, v in avaliacoesFilme[movies[i]].items():
        if v is None:
            venone.append(k)
            
    print(f"{movies[i]} - {venone}")

    for k in venone:
        del avaliacoesFilme[movies[i]][k]
        
    venone.clear()

Freddy x Jason - ['Claudia', 'Leonardo']
O Ultimato Bourne - []
Star Trek - ['Pedro', 'Janaina', 'Leonardo']
Exterminador do Futuro - []
Norbit - ['Pedro']
Star Wars - ['Leonardo']


In [16]:
avaliacoesFilme

{'Freddy x Jason': {'Ana': 2.5,
  'Marcos': 3.0,
  'Pedro': 2.5,
  'Adriano': 3.0,
  'Janaina': 3.0},
 'O Ultimato Bourne': {'Ana': 3.5,
  'Marcos': 3.5,
  'Pedro': 3.0,
  'Claudia': 3.5,
  'Adriano': 4.0,
  'Janaina': 4.0,
  'Leonardo': 4.5},
 'Star Trek': {'Ana': 3.0, 'Marcos': 1.5, 'Claudia': 3.0, 'Adriano': 2.0},
 'Exterminador do Futuro': {'Ana': 3.5,
  'Marcos': 5.0,
  'Pedro': 3.5,
  'Claudia': 4.0,
  'Adriano': 3.0,
  'Janaina': 5.0,
  'Leonardo': 4.0},
 'Norbit': {'Ana': 2.5,
  'Marcos': 3.5,
  'Claudia': 2.5,
  'Adriano': 2.0,
  'Janaina': 3.5,
  'Leonardo': 1.0},
 'Star Wars': {'Ana': 3.0,
  'Marcos': 3.0,
  'Pedro': 4.0,
  'Claudia': 4.5,
  'Adriano': 3.0,
  'Janaina': 3.0}}

---

In [17]:
avaliacoesFilme['Star Wars']

{'Ana': 3.0,
 'Marcos': 3.0,
 'Pedro': 4.0,
 'Claudia': 4.5,
 'Adriano': 3.0,
 'Janaina': 3.0}

In [18]:
avaliacoesFilme['Star Wars']['Ana']

3.0

In [19]:
getSimilares(avaliacoesFilme, 'Star Wars')

[(0.38742588672279304, 'Freddy x Jason'),
 (0.32037724101704074, 'O Ultimato Bourne'),
 (0.2989350844248255, 'Star Trek'),
 (0.29429805508554946, 'Norbit'),
 (0.252650308587072, 'Exterminador do Futuro')]

In [20]:
getSimilares(avaliacoesFilme, 'Norbit')

[(0.4494897427831781, 'Freddy x Jason'),
 (0.32037724101704074, 'Star Trek'),
 (0.29429805508554946, 'Star Wars'),
 (0.1918253663634734, 'Exterminador do Futuro'),
 (0.1886378647726465, 'O Ultimato Bourne')]

In [21]:
getRecomendacoesUsuario(avaliacoesFilme, 'Star Trek')

[(3.581097064761866, 'Janaina'),
 (3.2059731906295044, 'Pedro'),
 (2.936629402844435, 'Leonardo')]

___

In [22]:
euclidiana(avaliacoesUsuario, 'Pedro', 'Janaina')

0.32037724101704074

In [23]:
euclidiana(avaliacoesFilme, 'Star Wars', 'Star Trek')

0.2989350844248255

In [24]:
getSimilares(avaliacoesUsuario, 'Pedro')

[(0.5358983848622454, 'Claudia'),
 (0.4721359549995794, 'Ana'),
 (0.38742588672279304, 'Leonardo'),
 (0.38742588672279304, 'Adriano'),
 (0.3405424265831667, 'Marcos'),
 (0.32037724101704074, 'Janaina')]

In [25]:
getSimilares(avaliacoesFilme, 'Star Wars')

[(0.38742588672279304, 'Freddy x Jason'),
 (0.32037724101704074, 'O Ultimato Bourne'),
 (0.2989350844248255, 'Star Trek'),
 (0.29429805508554946, 'Norbit'),
 (0.252650308587072, 'Exterminador do Futuro')]

In [26]:
getRecomendacoesUsuario(avaliacoesUsuario, 'Leonardo')

[(3.457128694491423, 'Star Wars'),
 (2.778584003814924, 'Freddy x Jason'),
 (2.422482042361917, 'Star Trek')]

In [27]:
getRecomendacoesUsuario(avaliacoesFilme, 'Star Wars')

[(3.1667425234070894, 'Leonardo')]

---

In [28]:
def carregaMovieLens(path='Arquivos/ml-100k'):
    filmes = {}
    for linha in open(path + '/u.item'):
        (id, titulo) = linha.split('|')[0:2]
        filmes[id] = titulo
    # print(filmes)
    
    base = {}
    for linha in open(path + '/u.data'):
        (usuario, idfilme, nota, tempo) = linha.split('\t')
        base.setdefault(usuario, {})
        base[usuario][filmes[idfilme]] = float(nota)
    return base

In [29]:
def calculaItensSimilares(base):
    result = {}
    for item in base:
        notas = getSimilares(base, item)
        result[item] = notas
    return result

In [30]:
def getRecomendacoesItens (baseUsuario, similaridadeItens, usuario):
    notasUsuario = baseUsuario[usuario]
    notas = {}
    totalSimilaridade = {}
    for (item, nota) in notasUsuario.items():
        for (similaridade, item2) in similaridadeItens[item]:
            if item2 in notasUsuario: continue
            notas.setdefault(item2, 0)
            notas[item2] += similaridade * nota
            totalSimilaridade.setdefault(item2, 0)
            totalSimilaridade[item2] += similaridade
    rankings = [(score/totalSimilaridade[item], item) for item, score in notas.items()]
    rankings.sort(reverse=True)
    return rankings